In [1]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
%matplotlib inline
import numpy as np
import json


#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim

import optuna
from optuna.trial import Trial

import pickle


pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions
from multimodal.helper_functions import   reproducible_comp,  get_final_df
from multimodal.model_architecture import Generic1DCNN
reproducible_comp()

2023-10-09 20:17:42.251469: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 20:17:42.251521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 20:17:42.251545: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 20:17:42.259107: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 20:17:43.019385: W tensorflow/compiler/

In [2]:
file_path = "../../../data/processed_data/acc_inner.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_acc_inner, category_acc_inner, category_names, df_acc_inner = get_final_df(data, is_acc_data=1, is_multilabel=1)

file_path = "../../../data/processed_data/acc_outer.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_acc_outer, category_acc_outer, category_names, df_acc_outer = get_final_df(data, is_acc_data=1, is_multilabel=1)

merged_features_acc = np.concatenate((features_acc_inner,features_acc_outer), axis=0)
merged_category_acc = np.concatenate((category_acc_inner,category_acc_outer), axis=0)

labels = merged_category_acc.copy()
features = merged_features_acc.copy()
print("Features Shape: ", features.shape)
print("Labels Shape: ", labels.shape)

Features Shape:  (2680, 3)
Labels Shape:  (2680, 4)


In [3]:
max_length = 500

for i in range (len(features)):
    if(len(features[i][0]<max_length)):
        features[i][0]=np.pad(features[i][0], (0,max_length-len(features[i][0])), 'constant', constant_values=(0))
        
    if(len(features[i][1]<max_length)):
        features[i][1]=np.pad(features[i][1], (0,max_length-len(features[i][1])), 'constant', constant_values=(0))
    
    if(len(features[i][2]<max_length)):
        features[i][2]=np.pad(features[i][2], (0,max_length-len(features[i][2])), 'constant', constant_values=(0))

In [4]:
feat= np.zeros(len(features)*3*max_length).reshape(-1, 3*max_length)
for i in range(len(features)):
    feat[i]= np.concatenate((features[i][0],features[i][1], features[i][2]), axis=None)
    

print("Updated Features Shape: ", feat.shape)

Updated Features Shape:  (2680, 1500)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(feat, labels, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels)

In [6]:
X_train = X_train.reshape(-1, 1, feat.shape[1])
X_test = X_test.reshape(-1, 1, feat.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2412, 1, 1500)
(2412, 4)
(268, 1, 1500)
(268, 4)


In [7]:
from torch.utils.data import DataLoader, TensorDataset

# Convert your data to PyTorch tensors
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the objective function for hyperparameter optimization
def objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        hidden_channels = trial.suggest_int("hidden_channels", 16, 256)
        n_layers = trial.suggest_int("n_layers", 1, 7)
        kernel_size = trial.suggest_int("kernel_size", 3, 9)
        stride = trial.suggest_int("stride", 1, 7)
        padding = trial.suggest_int("padding", 1, 10)
        pooling_type = trial.suggest_categorical("pooling_type", ["max", "avg"])  
        use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False]) 


        # Split the data into training and validation sets
        x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(
            x_train, y_train, test_size=0.2, random_state=42
        )

        train_dataset = TensorDataset(x_train_split, y_train_split)
        valid_dataset = TensorDataset(x_valid_split, y_valid_split)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

        # Create the model with the suggested hyperparameters
        model = Generic1DCNN(input_channels, num_classes, n_layers, kernel_size, stride, padding,
                            hidden_channels, dropout_prob, pooling_type, use_batch_norm).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for inputs_valid, labels_valid in valid_loader:
                    inputs_valid, labels_valid = inputs_valid.to(device), labels_valid.to(device)
                    outputs_valid = model(inputs_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss
    


In [8]:
# Create an Optuna study with SQLite storage and TPE pruner
study = optuna.create_study(
    study_name="Case#6: Acceleration Merged",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),  
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 100  

# Perform hyperparameter optimization
study.optimize(objective, n_trials=num_trials, n_jobs=1)

[I 2023-10-09 22:32:09,906] A new study created in RDB with name: Case#6: Acceleration Merged


Epoch 10/50, Training Loss: 0.6596505661164561, Validation Loss: 0.6555318608880043
Epoch 20/50, Training Loss: 0.6296747173032453, Validation Loss: 0.6280902698636055
Epoch 30/50, Training Loss: 0.6228819797115941, Validation Loss: 0.6234875917434692
Epoch 40/50, Training Loss: 0.6147324404408855, Validation Loss: 0.6149635463953018


[I 2023-10-09 22:32:32,874] Trial 0 finished with value: 0.5986220613121986 and parameters: {'lr': 1.8253412438139013e-05, 'dropout_prob': 0.30603549511444855, 'hidden_channels': 80, 'n_layers': 3, 'kernel_size': 6, 'stride': 4, 'padding': 4, 'pooling_type': 'avg', 'use_batch_norm': False}. Best is trial 0 with value: 0.5986220613121986.


Epoch 50/50, Training Loss: 0.5992789672267053, Validation Loss: 0.5986220613121986


[I 2023-10-09 22:32:33,751] Trial 1 finished with value: 1.0 and parameters: {'lr': 0.000582978882533325, 'dropout_prob': 0.4362414036396269, 'hidden_channels': 248, 'n_layers': 6, 'kernel_size': 9, 'stride': 6, 'padding': 6, 'pooling_type': 'max', 'use_batch_norm': False}. Best is trial 0 with value: 0.5986220613121986.


...

Epoch 50/50, Training Loss: 0.1078051952344756, Validation Loss: 0.3907764405012131


[I 2023-10-10 00:45:49,894] Trial 98 finished with value: 1.0 and parameters: {'lr': 5.915109209376026e-05, 'dropout_prob': 0.3111944223840047, 'hidden_channels': 64, 'n_layers': 7, 'kernel_size': 9, 'stride': 2, 'padding': 3, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 45 with value: 0.260076941922307.


Trial failed with error: Calculated padded input size per channel: (7). Kernel size: (9). Kernel size can't be greater than actual input size
Epoch 10/50, Training Loss: 0.3862366435989257, Validation Loss: 0.37433938309550285
Epoch 20/50, Training Loss: 0.31530202540659136, Validation Loss: 0.3271140493452549
Epoch 30/50, Training Loss: 0.2646079322984142, Validation Loss: 0.29505973868072033
Epoch 40/50, Training Loss: 0.235446093303542, Validation Loss: 0.26540537364780903


[I 2023-10-10 00:46:53,047] Trial 99 finished with value: 0.26858755201101303 and parameters: {'lr': 9.65345922198704e-05, 'dropout_prob': 0.34043373377596814, 'hidden_channels': 52, 'n_layers': 7, 'kernel_size': 9, 'stride': 1, 'padding': 2, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 45 with value: 0.260076941922307.


Epoch 50/50, Training Loss: 0.20971207032280584, Validation Loss: 0.26858755201101303


In [9]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 5.254953458372508e-05, 'dropout_prob': 0.2838690767038667, 'hidden_channels': 63, 'n_layers': 7, 'kernel_size': 9, 'stride': 1, 'padding': 3, 'pooling_type': 'avg', 'use_batch_norm': True}
Best final validation loss:
0.260076941922307


In [10]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/acc_merged_best_configs.json"

# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

In [11]:
# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [14]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You